In [84]:
!pip install boto3

In [92]:
extracted_text="""Not a Memorial Hospital
Of Collier
Reg: PN/S/11011, Non-Profit
Contact: (999)-(888)-(1234)

Physician Hospital Discharge Summary

Provider: Mateo Jackson, Phd

Patient: John Doe

Provider's Pt ID: 00988277891

Patient Gender: Male

Attachment Control Number: XA/7B/00338763

Visit (Encounter)

Admitted: 07-Sep-2020

Discharged: 08-Sep-2020

Discharged to: Home with support services

Assessment

Reported Symptoms / History
of present illness:

35 yo M c/o stomach problems since 2 montsh ago. Patient
reports epigastric abdominal pain non-radiating. Pain is
described as gnawing and burning, intermitent lasting 1-2
hours, and gotten progressively worse. Antacids used to
alleviate pain but not anymore; nothing exhacerbates pain.
Pain unrelated to daytime or to meals. Patient denies
constipation or diarrhea. Patient denies blood in stool but
have noticed them darker. Patient also reports nausea.
Denies recent illness or fever. He also reports fatigue
since 2 weeks ago and bloating after eating.
Patient ID: NARH-36640
ROS: Negative except for above findings
Meds: Motrin once/week. Tums previously.
PMHX: Back pain and muscle spasms. No HX of surgery. NKDA.
FHX: Uncle has a bleeding ulcer.

Discharge

Discharge Studies Summary:
Some activity restrictions suggested, full course of
antibiotics, check back with physican in case of relapse,
strict diet

Social Hx: Smokes since 15 yo, 1/2-1 PPD. No recent EtOH
use. Denies illicit drug use. works on high elevation
construction. Fast food diet. Exercises 3-4 times/week but
stopped 2 weeks ago.

VS/3S/Q990-7550/1090001004290"""

In [122]:
input_text="""
You are a helpful assistant. Please extract the following details from the extracted_text & format the output as JSON using the keys

<details>
doctor_name: The doctor or provider's full name
provider_id: The doctor or provider's ID
patient_name: The patient's full name
patient_id: The patient's ID
patient_gender: The patient's gender
patient_age: The patient's age
admitted_date: Date the patient was admitted to the hospital
discharge_date: Date the patient was discharged from the hospital
discharged_to: where the patient was released or discharged to
drug_allergies: The patient's known drug allergies (NKDA)
patient_medical_history: The patient's medical history (PMHx)
family_medical_history: The patient's family medical history (FHx)
social_habits: The patient's social habits (Social HX)
</details>

<keys>
doctor_name
provider_id
patient_name
patient_id
patient_gender
patient_age
admitted_date
discharge_date
discharged_to
drug_allergies
patient_medical_history
family_medical_history
social_habits
</keys>

<extracted_text>
{extracted_text}
</extracted_text>
"""

In [123]:
import json
import boto3
client=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='{Put your AWS Access Key ID here}',aws_secret_access_key='{Put your AWS Secret Access Key here}')
inputText=input_text.format(extracted_text=extracted_text)
body_part=json.dumps({'inputText': inputText,
 'textGenerationConfig': {'maxTokenCount': 8192,
  'stopSequences': [],
  'temperature': 0,
  'topP': 1}})
response = client.invoke_model(
    body=body_part,
    contentType="application/json",
    accept="application/json",
    modelId='amazon.titan-text-express-v1'
)
output_text=json.loads(response['body'].read())['results'][0]['outputText']
print(output_text)

```tabular-data-json
{
    "rows": [
        {
            "doctor_name": "Mateo Jackson, Phd",
            "provider_id": "00988277891",
            "patient_name": "John Doe",
            "patient_id": "NARH-36640",
            "patient_gender": "Male",
            "admitted_date": "07-Sep-2020",
            "discharge_date": "08-Sep-2020",
            "discharged_to": "Home with support services",
            "drug_allergies": "NKDA",
            "patient_medical_history": "Back pain and muscle spasms. No HX of surgery. ",
            "family_medical_history": "Uncle has a bleeding ulcer.",
            "social_habits": "Smokes since 15 yo, 1/2-1 PPD. No recent EtOH use. Denies illicit drug use. works on high elevation construction. Fast food diet. Exercises 3-4 times/week but stopped 2 weeks ago."
        }
    ]
}
```


In [124]:
print(output_text.strip('```tabular-data-json'))


{
    "rows": [
        {
            "doctor_name": "Mateo Jackson, Phd",
            "provider_id": "00988277891",
            "patient_name": "John Doe",
            "patient_id": "NARH-36640",
            "patient_gender": "Male",
            "admitted_date": "07-Sep-2020",
            "discharge_date": "08-Sep-2020",
            "discharged_to": "Home with support services",
            "drug_allergies": "NKDA",
            "patient_medical_history": "Back pain and muscle spasms. No HX of surgery. ",
            "family_medical_history": "Uncle has a bleeding ulcer.",
            "social_habits": "Smokes since 15 yo, 1/2-1 PPD. No recent EtOH use. Denies illicit drug use. works on high elevation construction. Fast food diet. Exercises 3-4 times/week but stopped 2 weeks ago."
        }
    ]
}



In [127]:
json.loads(output_text.strip('```tabular-data-json'))['rows'][0]

{'doctor_name': 'Mateo Jackson, Phd',
 'provider_id': '00988277891',
 'patient_name': 'John Doe',
 'patient_id': 'NARH-36640',
 'patient_gender': 'Male',
 'admitted_date': '07-Sep-2020',
 'discharge_date': '08-Sep-2020',
 'discharged_to': 'Home with support services',
 'drug_allergies': 'NKDA',
 'patient_medical_history': 'Back pain and muscle spasms. No HX of surgery. ',
 'family_medical_history': 'Uncle has a bleeding ulcer.',
 'social_habits': 'Smokes since 15 yo, 1/2-1 PPD. No recent EtOH use. Denies illicit drug use. works on high elevation construction. Fast food diet. Exercises 3-4 times/week but stopped 2 weeks ago.'}

# **Prompt Improvement**

In [128]:
input_text="""
You are a helpful assistant. Please extract the following details from the extracted_text & format the output as JSON using the keys, return Only Json data within curly brackets, don't put any text other than the data

<details>
doctor_name: The doctor or provider's full name
provider_id: The doctor or provider's ID
patient_name: The patient's full name
patient_id: The patient's ID
patient_gender: The patient's gender
patient_age: The patient's age
admitted_date: Date the patient was admitted to the hospital
discharge_date: Date the patient was discharged from the hospital
discharged_to: where the patient was released or discharged to
drug_allergies: The patient's known drug allergies (NKDA)
patient_medical_history: The patient's medical history (PMHx)
family_medical_history: The patient's family medical history (FHx)
social_habits: The patient's social habits (Social HX)
</details>

<keys>
doctor_name
provider_id
patient_name
patient_id
patient_gender
patient_age
admitted_date
discharge_date
discharged_to
drug_allergies
patient_medical_history
family_medical_history
social_habits
</keys>

<extracted_text>
{extracted_text}
</extracted_text>
"""

In [129]:
import json
import boto3
client=boto3.client('bedrock-runtime',region_name='us-east-1',aws_access_key_id='{Put your AWS Access Key ID here}',aws_secret_access_key='{Put your AWS Secret Access Key here}')
inputText=input_text.format(extracted_text=extracted_text)
body_part=json.dumps({'inputText': inputText,
 'textGenerationConfig': {'maxTokenCount': 8192,
  'stopSequences': [],
  'temperature': 0,
  'topP': 1}})
response = client.invoke_model(
    body=body_part,
    contentType="application/json",
    accept="application/json",
    modelId='amazon.titan-text-express-v1'
)
output_text=json.loads(response['body'].read())['results'][0]['outputText']
print(output_text)

```
{
    "doctor_name": "Mateo Jackson, Phd",
    "provider_id": "00988277891",
    "patient_name": "John Doe",
    "patient_id": "NARH-36640",
    "patient_gender": "Male",
    "admitted_date": "07-Sep-2020",
    "discharge_date": "08-Sep-2020",
    "discharged_to": "Home with support services",
    "drug_allergies": "NKDA",
    "patient_medical_history": "Back pain and muscle spasms. No HX of surgery.",
    "family_medical_history": "Uncle has a bleeding ulcer.",
    "social_habits": "Smokes since 15 yo, 1/2-1 PPD. No recent EtOH use. Denies illicit drug use. works on high elevation construction. Fast food diet. Exercises 3-4 times/week but stopped 2 weeks ago."
}
```


In [130]:
print(output_text.strip('```'))


{
    "doctor_name": "Mateo Jackson, Phd",
    "provider_id": "00988277891",
    "patient_name": "John Doe",
    "patient_id": "NARH-36640",
    "patient_gender": "Male",
    "admitted_date": "07-Sep-2020",
    "discharge_date": "08-Sep-2020",
    "discharged_to": "Home with support services",
    "drug_allergies": "NKDA",
    "patient_medical_history": "Back pain and muscle spasms. No HX of surgery.",
    "family_medical_history": "Uncle has a bleeding ulcer.",
    "social_habits": "Smokes since 15 yo, 1/2-1 PPD. No recent EtOH use. Denies illicit drug use. works on high elevation construction. Fast food diet. Exercises 3-4 times/week but stopped 2 weeks ago."
}

